In [2]:
import sys
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
from functions import *

In [3]:
bpath = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/behavioral/'

def get_resids(data, var):
    import statsmodels.formula.api as smf
    model = smf.ols(var + '~ interview_age + C(sex)', data=data)
    model = model.fit()
    resid = model.resid
    return resid

def add_resid(data, var):
    data[var + "_r"] = get_resids(data, var)
    return data

In [4]:
# NIH w PC --------------------------------------
nih_toolbox = (readit('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/nda18_w_nc_pca.csv')
               .rename(columns={'src_subject_id': 'ID', 'Age':'interview_age', 'Female':'sex'}))
nih_toolbox['ID'] = nih_toolbox['ID'].str.replace('_', '')
nih_vars = ['PicVocab','Flanker','List','CardSort','Pattern','Picture','Reading','RAVLT','LMT','pc1_new','pc2_new','pc3_new']
nih_toolbox = nih_toolbox[['ID', 'interview_age', 'sex'] + nih_vars]
               
for i in nih_vars: nih_toolbox = add_resid(nih_toolbox, i)
nih_sub_scales = [nih_vars + "_r" for nih_vars in nih_vars]
nih_toolbox = nih_toolbox[['ID'] + nih_sub_scales]
nih_toolbox.to_csv(bpath + 'nih_baseline.csv', index=False)


# Stroop --------------------------------------
stroop_beh = (readit('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/data_ABCD_PAPER_Stroop.csv')
              .rename(columns={'subjectkey': 'ID', 'interview_age.x':'interview_age', 'sex.x':'sex'}))
stroop_beh['ID'] = stroop_beh['ID'].str.replace('_', '')

stroop_vars = ['Stroop_interf_acc_all', 'Stroop_interf_RT_all', 'Stroop_interf_acc_Eq',
               'Stroop_interf_RT_Eq', 'Stroop_interf_acc_MC', 'Stroop_interf_RT_MC', 'Happy_RT_Eq', 
               'Angry_RT_Eq', 'Happy_Acc_Eq', 'Angry_Acc_Eq','HappyMinusAngry_RT_Eq', 'HappyMinusAngry_Acc_Eq',
               'Stroop_interf_RT_angryMC', 'Stroop_interf_RT_happyMC', 'Stroop_interf_acc_angryMC', 'Stroop_interf_acc_happyMC']
            
stroop_beh = stroop_beh[['ID', 'interview_age', 'sex'] + stroop_vars]

for i in stroop_vars: stroop_beh = add_resid(stroop_beh, i)
stroop_sub_scales = [stroop_vars + "_r" for stroop_vars in stroop_vars]
stroop_beh = stroop_beh[['ID'] + stroop_sub_scales]
stroop_beh.to_csv(bpath + 'stroop_baseline.csv', index=False)


# SST  --------------------------------------
sst = readtxt(bpath + 'abcd_sst02.txt').query('eventname == "baseline_year_1_arm_1"')
sst['ID'] = sst['src_subject_id'].str.replace('_', '', regex=True)
sst = sst.drop(['collection_id','abcd_sst02_id','dataset_id','subjectkey','src_subject_id',
'interview_date','eventname','collection_title', 'tfmri_sst_beh_visitid'], axis=1)
sst_vars = sst.iloc[:, 2:97].columns.to_list()
sst = sst[['ID', 'interview_age', 'sex'] + sst_vars]
#for i in sst_vars: sst = add_resid(sst, i)
#sst_sub_scales = [stroop_vars + "_r" for sst__vars in sst_vars]
#sst = sst[['ID'] + sst_sub_scales]
sst.to_csv(bpath + 'sst_baseline.csv', index=False)


# Pearson (Matrix Reasoning) --------------------------------------
pearson = readtxt(bpath + 'abcd_ps01.txt').query('eventname == "baseline_year_1_arm_1"')
pearson['ID'] = pearson['src_subject_id'].str.replace('_', '', regex=True)
pearson = pearson[['ID', 'interview_age', 'sex', 'pea_wiscv_trs']]

pearson[['interview_age', 'pea_wiscv_trs']] = pearson[['interview_age', 'pea_wiscv_trs']].apply(pd.to_numeric, errors='coerce')
pearson_final = (add_resid(pearson, 'pea_wiscv_trs')
           .drop(['interview_age', 'sex','pea_wiscv_trs'], axis=1)
           .rename({'pea_wiscv_trs_r':'matrix_r'}, axis=1))

pearson_final.to_csv(bpath + 'pearson_baseline.csv', index=False)